In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import pickle
import json
import requests
import tweepy

key = 'B81Sj4ifYRstLZGMyMDM0roG3'
secret = 'c1Rg4vdyl2mjMEC7WKDsVLTz2IlqTHy1yezn71Y9cw3LgZaxAW'
token = 'AAAAAAAAAAAAAAAAAAAAAD6eWQEAAAAABtT80i7lkSemhYsuOIjy4O4xLQs%3DkfsByhRoXQ9odyT51sECR1vXxOLKxdtOON2Ij4OODdAt8jTcGY'

api = tweepy.Client(bearer_token=token, consumer_key=key, consumer_secret=secret, 
              return_type=requests.Response,
              wait_on_rate_limit=True)

In [2]:
# Returns the followers/following of the center node in the network (because these aren't in the pickle files)
def getCenterNodeFollows(file):
    centerNode = file.replace('.pkl','').replace('edgelist_2_deg_','').replace('edgelist_mentions_','').replace('/','').replace('human','').replace('bot','').replace('mentions','')
        
    r = api.get_user(id=centerNode, user_fields='public_metrics')
    user = r.json()

    followers = user['data']['public_metrics']['followers_count']
    following = user['data']['public_metrics']['following_count']
    
    return followers, following

In [26]:
# Takes in a pickle object and returns in/out degree centralities, 
# clustering coefficient, local homophily metrics, and PageRank for account ID.
def node_calculations(file, ID):
    full_list = pd.read_pickle(file)
    
    if 'edge_weight' in full_list.columns:
        weightlist = full_list['edge_weight']
    else:
        weightlist = [1]*len(full_list['originating_id'])
    
    edgelist = pd.DataFrame({
        "source": full_list['originating_id'],
        "target": full_list['receiving_id'],
        "weight": weightlist,
    })
    
    # store originating ID followers
    attr = dict()
    for i, row in full_list.iterrows():
        if not pd.isna(row['originating_following_count']):
            attr[row['originating_id']] = {'in': row['originating_follower_count'], 
                                           'out': row['originating_following_count']}
        if not pd.isna(row['receiving_following_count']):
            attr[row['receiving_id']] = {'in': row['receiving_follower_count'], 
                                         'out': row['receiving_following_count']}
            
    IDfollowers, IDfollowing = getCenterNodeFollows(file)
    attr[ID] = {'in': IDfollowers, 'out': IDfollowing}
    # NOTE: nothing in "attr" represents the degrees of nodes in OUR network; the attributes represent
    # the degrees of nodes in the FULL network
    
    #print(attr)
            
    G = nx.from_pandas_edgelist(edgelist, create_using=nx.DiGraph(), edge_attr=True)
    nx.set_node_attributes(G, attr)
    
    nodes = list(G.nodes)
    
    clustercoeff = nx.clustering(G, ID, weight='weight')
    pagerank = nx.pagerank(G, weight='weight')[ID]
    
    # compute "local homophily" for ID's degree structure in two different ways
    indeg = len(list(G.predecessors(ID)))
    outdeg = len(list(G.successors(ID)))
    
    outdeg_Assort1 = 0
    outdeg_Assort2 = 0
    indeg_Assort1 = 0
    indeg_Assort2 = 0
    
    if outdeg != 0:
        for node in G.neighbors(ID):
            nodeOutdeg = attr[node]['out']
            outdeg_Assort1 += nodeOutdeg
            if nodeOutdeg >= IDfollowing/2 and nodeOutdeg <= IDfollowing*2:
                outdeg_Assort2 += 1
                
        outdeg_Assort1 = outdeg_Assort1 / (outdeg*IDfollowing)
        outdeg_Assort2 = outdeg_Assort2 / outdeg

    if indeg != 0: 
        for node in G.predecessors(ID):    
            nodeIndeg = attr[node]['in']
            indeg_Assort1 += nodeIndeg
            if nodeIndeg >= IDfollowers/2 and nodeIndeg <= IDfollowers*2:
                indeg_Assort2 += 1
    
        indeg_Assort1 = indeg_Assort1 / (indeg*IDfollowers)
        indeg_Assort2 = indeg_Assort2 / indeg
    
    
    return (IDfollowers, IDfollowing, clustercoeff, pagerank, indeg_Assort1, outdeg_Assort1, \
            indeg_Assort2, outdeg_Assort2)

In [27]:
file = 'mentions/human/edgelist_mentions_456361810.pkl'
ID = 456361810
full_list = pd.read_pickle(file)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #print(full_list)
indeg, outdeg, clustercoeff, pagerank, indeg_A1, outdeg_A1, indeg_A2, outdeg_A2 = node_calculations(file, ID)
print([clustercoeff, pagerank, indeg_A1, outdeg_A1, indeg_A2, outdeg_A2])

[0.0018591559380531392, 0.161441094880482, 216.7860696517413, 0, 0.16666666666666666, 0]
